In [1]:
print("Activating ipynb")
import os 
import numpy as np
import pandas as pd

games_folder_path = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/Year/Games/"
years = [x for x in range(2012,2022)]
for year in years:
    exec(f'games_filename_{year} = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/Year/Games/{year}.csv"')
    # exec(f'games_filename_{year} = {os.path.join(games_folder_path, f"{year}.csv")}')
all_games = os.path.join("C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/games_sorted.csv")
all_stats = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/stats_sorted.csv"
print(all_games)

Activating ipynb
C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/games_sorted.csv


https://www.youtube.com/watch?v=k7hSD_-gWMw

In [2]:
for year in years:
    exec(f'results_{year} = pd.read_csv(games_filename_{year}, index_col = False)')
    
all_results_raw = pd.read_csv(all_games)
all_stats_raw = pd.read_csv(all_stats)
all_results = all_results_raw

In [3]:
for year in years:
    exec(f'results_{year}["homeWin"] = results_{year}["homeTeamScore"] > results_{year}["awayTeamScore"]')
    exec(f'y_true_{year} = results_{year}["homeWin"].values')  

all_results['homeWin'] = all_results['homeTeamScore'] > all_results['awayTeamScore']
y_true_all = all_results['homeWin'].values
all_results[:5]

,gameId,year,round,date,venue,startTime,attendance,homeTeam,homeTeamScore,awayTeam,awayTeamScore,rainfall,homeWin
0,2012R101,2012,R1,24-Mar-12,Stadium Australia,7:20 PM,38203,Greater Western Sydney,37,Sydney,100,0.0,False
1,2012R102,2012,R1,29-Mar-12,M.C.G.,7:45 PM,78285,Richmond,81,Carlton,125,0.0,False
2,2012R103,2012,R1,30-Mar-12,M.C.G.,7:50 PM,78466,Hawthorn,137,Collingwood,115,0.0,True
3,2012R104,2012,R1,31-Mar-12,M.C.G.,1:45 PM,33473,Melbourne,78,Brisbane Lions,119,0.6,False
4,2012R105,2012,R1,31-Mar-12,Carrara,3:45 PM,12790,Gold Coast,68,Adelaide,137,0.0,False


## Feature Building  
HomeTeam, AwayTeam, Venue, Rainfall, Travel, TeamChangeNumber, TeamChangeGames, LastWeekResult, LastEncounterResult, WinStreak
Form(Last5Games), TeamPlayerTotalRankingScore, BreakBetweenGames, LadderPosition

In [9]:
## Remove uneeded fields
results_clean_2012 = results_2012
results_clean_2012.describe()

,year,attendance,homeTeamScore,awayTeamScore,rainfall
count,207.0,207.000000,207.000000,207.000000,207.000000
mean,2012.0,32747.942029,96.376812,87.347826,2.252174
std,0.0,18335.613605,30.488149,29.316564,5.400394
min,2012.0,6696.000000,37.000000,31.000000,0.000000
25%,2012.0,18922.500000,77.000000,65.500000,0.000000
50%,2012.0,30172.000000,94.000000,86.000000,0.200000
75%,2012.0,41277.000000,114.000000,105.000000,2.000000
max,2012.0,99683.000000,193.000000,180.000000,38.400000


In [7]:
results_clean_2012 = results_2012
results_clean_2012.columns
results_clean_2012 = results_clean_2012.drop(['year', 'date', 'startTime', 'attendance'], axis=1)
results_clean_2012.columns

a_2012 = results_clean_2012

## Build Y value 
Result of game based on Home Team (Win = 2, Draw = 1, Lose = 0)

In [8]:
def homeWin(row):
    homeScore = row["homeTeamScore"]
    awayScore = row["awayTeamScore"]
    row['homeWin'] = 2 if homeScore > awayScore else 0 if homeScore < awayScore else 1 
        
    return row
  
a_2012 = a_2012.apply(homeWin, axis=1)
y_true_2012 = a_2012['homeWin']


In [33]:
def show_dtypes(df):
    for index in range(len(df.dtypes)):
        print(f'{df.columns[index]} -> {df.dtypes[index]}' )

show_dtypes(a_2012)
a_2012.isnull().sum()

gameId -> object
round -> object
venue -> object
homeTeam -> object
homeTeamScore -> int64
awayTeam -> object
awayTeamScore -> int64
rainfall -> float64
homeWin -> int64


gameId           0
round            0
venue            0
homeTeam         0
homeTeamScore    0
awayTeam         0
awayTeamScore    0
rainfall         0
homeWin          0
dtype: int64

## Add Previous Game Result
Set default dict to store results for each team
Only a boolean value, draws only account for ~1% of all games 
Default to false
If the last game was a win set to true
# TODO 
add draws in and see if there is a difference

In [34]:
a_2012["homeTeamLastWin"] = False
a_2012["awayTeamLastWin"] = False

from collections import defaultdict
won_last = defaultdict(bool)

def teamLastWin(row):
    home_team = row["homeTeam"]
    away_team = row["awayTeam"]
    row["homeTeamLastWin"] = won_last[home_team]
    row["awayTeamLastWin"] = won_last[away_team]
    
    won_last[home_team] = row["homeWin"] == "Win"
    won_last[away_team] = row["homeWin"] == "Loss"
    
    return row
  
a_2012 = a_2012.apply(teamLastWin, axis=1)

## One Hot Encoding
The tree models used for feature engineering cannot take in string inputs
Value encoding can be used to change teams to int values, but then the model will read this as distance i.e. the team with value 1 will be read as being closer to the team with value 2 than the team with value 18. 
One Hot Encoding creates a bool variable for each team which is 1 when the team is playing.

# TODO
See if the model performs better with the simple label encoding. 
One hot encoding creates an issue with decision trees as it creates lots of "empty" splits, there will now be 34 variables which the model will split on 0, which can skew the model.

In [35]:
# One Hot Encoding Teams
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoding = LabelEncoder()
encoding.fit(all_results["homeTeam"].values)

home_teams_2012 = encoding.transform(a_2012["homeTeam"].values)
away_teams_2012 = encoding.transform(a_2012["awayTeam"].values)

all_teams_2012 = np.vstack([home_teams_2012, away_teams_2012]).T
all_teams_2012[:5], all_teams_2012.shape

oneHot = OneHotEncoder()
X_teams_2012 = oneHot.fit_transform(all_teams_2012).todense()
X_teams_2012.shape

(207, 36)

## One Hot Encoding Venues
Same issues as above, but simplified slightly as there are only 23 stadiums

In [36]:
# One Hot Encoding Venues
encoding = LabelEncoder()
encoding.fit(all_results["venue"].values)
venues = encoding.transform(a_2012['venue'].values)

all_venues = np.vstack(venues).T
venues = venues.reshape(-1,1)
venues[:5], venues.shape

oneHot = OneHotEncoder()
X_venues = oneHot.fit_transform(venues).todense()
X_venues = np.asarray(X_venues)
X_venues.shape

(207, 16)

Computed Variables
These variables did not appear in the original data set but were added later
Travel Data: Used to determine the impact of travel on teams
             Determined by comparing the venue of the upcoming match to the teams home location
Break: Finds the break between a teams last and next game
Team Changes: The number of players that are different in the from the previous match
              The number of games lost/gained from these changes
              The expected score lost/gained from the incoming players
              # TODO split games out and games in into separate variables
Team Composition: The average/total score of all the selected players
                  Done this way to reduce number of features
Previous 5 games result: Totals each teams previous games results

             

In [37]:
import AFL_Stats.afl_stats as st
def travel(row):
    row['awayTravel'] = st.team_travel(row['awayTeam'], row['venue'])
    row['homeTravel'] = st.team_travel(row['homeTeam'], row['venue'])
    return row
a_2012 = a_2012.apply(travel, axis=1)

a_2012[:10]

,gameId,round,venue,homeTeam,homeTeamScore,awayTeam,awayTeamScore,rainfall,homeWin,homeTeamLastWin,awayTeamLastWin,awayTravel,homeTravel
0,2012R101,R1,Stadium Australia,Greater Western Sydney,37,Sydney,100,0.0,0,False,False,0,0
1,2012R102,R1,M.C.G.,Richmond,81,Carlton,125,0.0,0,False,False,0,0
2,2012R103,R1,M.C.G.,Hawthorn,137,Collingwood,115,0.0,2,False,False,0,0
3,2012R104,R1,M.C.G.,Melbourne,78,Brisbane Lions,119,0.6,0,False,False,0,0
4,2012R105,R1,Carrara,Gold Coast,68,Adelaide,137,0.0,0,False,False,3,0
5,2012R106,R1,Subiaco,Fremantle,105,Geelong,101,0.0,2,False,False,3,0
6,2012R107,R1,Docklands,North Melbourne,102,Essendon,104,0.6,0,False,False,0,0
7,2012R108,R1,Docklands,Western Bulldogs,87,West Coast,136,0.0,0,False,False,4,0
8,2012R109,R1,Football Park,Port Adelaide,89,St Kilda,85,0.0,2,False,False,2,0
9,2012R201,R2,Gabba,Brisbane Lions,63,Carlton,154,0.0,0,False,False,3,0


In [29]:
# Replace Teams and venues with OHE values
a_2012_ohe = a_2012.drop(['homeTeam','awayTeam','venue','homeWin', 'homeTeamScore','awayTeamScore', 'round'], axis=1)
a_2012_ohe.columns
a_2012_ohe = np.hstack([a_2012_ohe, X_teams_2012, X_venues])

a_2012_ohe

matrix([[0.0, False, False, ..., 0.0, 0.0, 0.0],
        [0.0, False, False, ..., 0.0, 0.0, 0.0],
        [0.0, False, False, ..., 0.0, 0.0, 0.0],
        ...,
        [0.0, False, False, ..., 0.0, 0.0, 0.0],
        [1.0, False, False, ..., 0.0, 0.0, 0.0],
        [8.4, False, False, ..., 0.0, 0.0, 0.0]], dtype=object)

In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier(random_state=26)
y_true_2012 = np.asarray(y_true_2012)
a_2012_ohe_in = np.asarray(a_2012_ohe)
scores = cross_val_score(clf, a_2012_ohe_in, y_true_2012)

print("F1: {0:0.4f}".format(np.mean(scores)))

F1: 0.6333


C:\Users\Craig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


# Baseline
Home team wins

In [4]:
for year in years:
    exec(f'n_games_{year} = results_{year}["homeWin"].count()')
    exec(f'n_homewins_{year} = results_{year}["homeWin"].sum()')
    exec(f'win_percentage_{year} = (n_homewins_{year} / n_games_{year}) * 100')
    exec(f'win_percentage = (n_homewins_{year} / n_games_{year}) * 100')

    # print(f'Homewins in {year}: {win_percentage}:')
    exec(f'print(win_percentage)')

n_games_total = all_results['homeWin'].count()
n_homewins_total = all_results['homeWin'].sum()
win_percentage_total = (n_homewins_total / n_games_total) * 100

print("Homewins percentage total: {0:.1f}%".format(win_percentage_total))


55.55555555555556
55.55555555555556
56.52173913043478
52.42718446601942
59.42028985507246
58.454106280193244
54.589371980676326
57.00483091787439
56.17283950617284
51.690821256038646
Homewins percentage total: 55.7%


In [5]:
from sklearn.metrics import f1_score
for year in years:
    exec(f'y_pred_{year} = [1] * len(y_true_{year})')
y_pred_all = [1] * len(y_true_all)

print("F1 Score 2012: {:.4f}".format(f1_score(y_true_2012, y_pred_2012, pos_label=None, average='weighted')))
print("F1 Score 2013: {:.4f}".format(f1_score(y_true_2013, y_pred_2013, pos_label=None, average='weighted')))
print("F1 Score 2014: {:.4f}".format(f1_score(y_true_2014, y_pred_2014, pos_label=None, average='weighted')))
print("F1 Score 2015: {:.4f}".format(f1_score(y_true_2015, y_pred_2015, pos_label=None, average='weighted')))
print("F1 Score 2016: {:.4f}".format(f1_score(y_true_2016, y_pred_2016, pos_label=None, average='weighted')))
print("F1 Score 2017: {:.4f}".format(f1_score(y_true_2017, y_pred_2017, pos_label=None, average='weighted')))
print("F1 Score 2018: {:.4f}".format(f1_score(y_true_2018, y_pred_2018, pos_label=None, average='weighted')))
print("F1 Score 2019: {:.4f}".format(f1_score(y_true_2019, y_pred_2019, pos_label=None, average='weighted')))
print("F1 Score 2020: {:.4f}".format(f1_score(y_true_2020, y_pred_2020, pos_label=None, average='weighted')))
print("F1 Score 2021: {:.4f}".format(f1_score(y_true_2021, y_pred_2021, pos_label=None, average='weighted')))
print("F1 Score Total: {:.4f}".format(f1_score(y_true_all, y_pred_all, pos_label=None, average='weighted')))

F1 Score 2012: 0.3968
F1 Score 2013: 0.3968
F1 Score 2014: 0.4082
F1 Score 2015: 0.3606
F1 Score 2016: 0.4430
F1 Score 2017: 0.4313
F1 Score 2018: 0.3855
F1 Score 2019: 0.4139
F1 Score 2020: 0.4041
F1 Score 2021: 0.3523
F1 Score Total: 0.3989


In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier(random_state=26)

X_prevWins = a_2012[["homeTeamLastWin", "awayTeamLastWin"]].values
scores = cross_val_score(clf, X_prevWins, y_true_2012)

print("F1: {0:0.4f}".format(np.mean(scores)))

F1: 0.5796


In [12]:
## need to look at previous years games

last_winner = defaultdict(bool)

def last_encounter_winner(row):
    home_team = row['homeTeam']
    away_team = row['awayTeam']

    teams = tuple(sorted([home_team, away_team]))
    result = True if last_winner[teams] == row['homeTeam'] else False

    winner = row['homeTeam'] if row['homeWin'] else row['awayTeam']
    last_winner[teams] = winner

    row["homeTeamWonLast"] = result
    return row

results_2012 = results_2012.apply(last_encounter_winner, axis=1)

In [82]:
# One Hot Encoding Teams
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoding = LabelEncoder()
encoding.fit(all_results["homeTeam"].values)


home_teams_2012 = encoding.transform(results_2012["homeTeam"].values)
away_teams_2012 = encoding.transform(results_2012["awayTeam"].values)

all_teams_2012 = np.vstack([home_teams_2012, away_teams_2012]).T
all_teams_2012[:5], all_teams_2012.shape

oneHot = OneHotEncoder()
X_teams_2012 = oneHot.fit_transform(all_teams_2012).todense()
X_teams_2012.shape

matrix([[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]])

In [127]:
# One Hot Encoding Venues
encoding = LabelEncoder()
encoding.fit(all_results["venue"].values)

venues = encoding.transform(results_2012['venue'].values)

all_venues = np.vstack(venues).T
venues = venues.reshape(-1,1)
venues[:5], venues.shape

oneHot = OneHotEncoder()
X_venues = oneHot.fit_transform(venues).todense()
X_venues = np.asarray(X_venues)
X_venues.shape

(207, 16)

In [133]:
clf = DecisionTreeClassifier()
scores_venues = cross_val_score(clf, results_2012["rainfall"], y_true_2012)
print(scores_venues)
print("F1: {0:0.4f}".format(np.mean(scores_venues)))

[nan nan nan nan nan]
F1: nan


C:\Users\Craig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Craig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Craig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Pyt

In [96]:
X_teams_2012.shape
X_teams_2012 = np.asarray(X_teams_2012)
print("Home:", X_teams_2012[0,:18])
print("Away", X_teams_2012[0,18:])

Home: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Away [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [76]:
X_2012 = np.hstack

In [97]:
clf_t = DecisionTreeClassifier(random_state=26)
scores_teams = cross_val_score(clf_t, X_teams_2012, y_true_2012)

print("F1: {0:0.4f}".format(np.mean(scores_teams)))

F1: 0.6619


In [100]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
parameter_space = {
    "max_features": [2,10,50,'auto'],
    "n_estimators": [50,100,200],
    "criterion": ["gini", "entropy"],
    "min_samples_leaf": [1,2,4,6]
}

rf = RandomForestClassifier(random_state=26)
grid = GridSearchCV(rf, parameter_space)
grid.fit(X_teams_2012, y_true_2012)
# scores_rf_teams = cross_val_score(rf, X_teams_2012, y_true_2012)
print("F1: {0:0.4f}".format(np.mean(grid.best_score_)))
# print(grid.best_estimator_)

F1: 0.7152
RandomForestClassifier(criterion='entropy', max_features=10, n_estimators=50,
                       random_state=26)


C:\Users\Craig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
120 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Craig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Craig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packag

In [51]:
mod_2012_results = a_2012[["homeTeamLastWin", "awayTeamLastWin"]]
X_2012_ref = np.hstack([a_2012, X_teams_2012])
X_2012_ref.shape    

(207, 46)

In [109]:
cl = DecisionTreeClassifier(random_state=26)
scores_all = cross_val_score(cl, X_2012_ref, y_true_2012)

print("F1: {0:0.4f}".format(np.mean(scores_all)))

F1: 0.7055
